In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.linalg import orthogonal_procrustes
from sklearn.metrics import mean_squared_error
from helpers_pmf import *
from helpers_similarity import *
from helpers_optimization import *
from scipy.optimize import minimize

In [88]:
mu = 0
sigma_u = 1
sigma_v = 1
sigma = 0
d_dim = 2
n_users = 20
n_movies = 40

Calculate the difference between each pair of elements in the 2D array X. Then, generate the probability matrix by applying the logistic function $$P(x) = \frac{e^{x}}{1 + e^{x}}$$ element-wise to the difference matrix, where x is the difference between two elements :

In [89]:
#change prod_matrix to P (same notation)
#change diff to X (like in the equation)
def generate_P_BT_Luce(X): 
    
    diff = np.subtract.outer(X, X) 
    diff_3D= np.array([diff[i, :, i, :] for i in range(n_users)])
    prob_matrix = np.exp(diff_3D) / (1 + np.exp(diff_3D))
    
    return prob_matrix, diff_3D


Generate pairwise comparison data $Y_{ijk} = \pm 1$ for each user and item. The output Y is a $3D$ tensor with the shape $(n\_users, n\_items, n\_items)$. Each entry $Y[i,j,k]$ corresponds to whether user i prefers item j over item k. For example, line 0 of Y corresponds to the pairwise comparisons of user 0 with all items: $Y[0,0,:]$ represents whether user 0 prefers item 0 over all other items $(item\_0, item\_1, item\_2, etc..)$, and so on.

In [90]:
def pairwise_comparisons(prob_matrix):
    
    Y = np.random.binomial(n=1, p=prob_matrix, size=prob_matrix.shape) 
    Y = np.where(Y == 0, -1, Y)
    
    return Y

In [91]:
U, V, X = generate_U_V_R(mu, sigma_u, sigma_v, sigma, d_dim, n_users, n_movies)

In [92]:
prob_matrix, diff = generate_P_BT_Luce(X)
Y = pairwise_comparisons(prob_matrix)

#print('X :\n ',X)
#print('Y :\n ',Y)

In [93]:
def hinge_loss(x):
    
    return np.maximum(0, 1 - x)**2 

In [94]:
def frob(X, Y, N):
    
    return np.linalg.norm(X - Y, 'fro')/np.sqrt(N)

In [95]:
def loss(U, Y, V, lambda_reg):
    U=U.reshape((d_dim, n_users))
    n, m, _ = Y.shape
    loss = 0
    for i in range(n):
        for j in range(m):
            for k in range(j):
                if Y[i, j, k] != 0:
                    v = V[:,j] - V[:,k]
                    x = np.dot(U[:,i], v)
                    loss += hinge_loss(Y[i, j, k] * x)
    reg = lambda_reg * np.linalg.norm(U, 'fro')#np.sum(W**2)
    return loss + reg

In [ ]:
lambda_reg = 0.1
U0 = np.random.normal(mu, sigma_u, d_dim*n_users)#initialize
res = minimize(loss, U0, args=(Y, V, lambda_reg), method='L-BFGS-B') #method='L-BFGS-B' ? # read abt it and see how it can improve result, local minima
U_res = res.x.reshape(U.shape)
print(U, U_res)
print(frob(U, U_res, U.shape[1]))